In [156]:
%load_ext autoreload
%autoreload 2
import os
import glob
import random
#from iucrime.records import Record

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [161]:
path = '../../data/text'
files = glob.glob(os.path.join(path, '*'))

Let's just pick one data file and take a look

In [162]:
fname = random.choice(files)
fname

'../../data/text/2-16-16.txt'

In [163]:
print(open(fname).read())

Indiana University, Bloomington

Police Department
daily crime log report
from 02/16/2016 00:00 to 02/16/2016 23:59
date reported: 02/16/16 - TUE at 13:13
general location:

report #:

160445

WOODBURN HALL - On Campus

date occurred from: 02/12/16 - FRI at 17:00
date occurred to:

02/16/16 - TUE at 13:00

incident/offenses:

VANDALISM, MISCHIEF

disposition: CLOSED CASE- NO ARREST
modified date: 02/17/16 - WED at 17:18
date reported: 02/16/16 - TUE at 15:19
general location:

report #:

IUPD STATION - On Campus

date occurred from: 02/01/16 - MON at 12:00
date occurred to:

02/01/16 - MON at 13:00

incident/offenses:

HARASSMENT/INTIMIDATION

disposition: OPEN CASE
modified date: 02/17/16 - WED at 16:12

160446




In [164]:
import re
from pprint import pprint
import datetime

class Record():
    """ class for keeping the daily IU police log """

    INFO_PATTERN = re.compile('^(location|incident|disposition|event #|report #)\s*:\s*(.+)', re.IGNORECASE)
    TIME_PATTERN = re.compile('^date and time occurred.*from.*to', re.IGNORECASE)
    
    def __init__(self, content):
        self.info = {} 
        
        self.parse(content)

    def parse(self, content):
        line_itr = iter(content.splitlines())
        for line in line_itr:
            m = re.search(Record.INFO_PATTERN, line)
            if m:
                key = m.group(1).lower().replace('#', 'number')
                val = m.group(2).lower()
                self.info[key] = val
                continue
                
            if re.match(Record.TIME_PATTERN, line):
                line = next(line_itr)
                t_from, t_to = self.parse_from_to_time(line)
                self.info['from datetime'] = t_from
                self.info['to datetime'] = t_to
                
    def parse_from_to_time(self, timestr):
        fromdate, fromtime, todate, totime = timestr.split('-')
        fromtime = fromtime.split('at')[1].strip()
        totime = totime.split('at')[1].strip()
        t_from = datetime.datetime.strptime(' '.join([fromdate.strip(), fromtime]), '%m/%d/%y %H:%M')
        t_to = datetime.datetime.strptime(' '.join([todate.strip(), totime]), '%m/%d/%y %H:%M')
        return (t_from, t_to)
                
r = Record(open(fname).read())
assert(r.info['location'] == 'indiana memorial union')
assert(r.info['incident'] == 'all other offenses - trespass & malicious trespass')
pprint(r.info)

KeyError: 'location'